## MDP Para el juego Black Jack (21)

Se intentará resolver a continuación una versión simplificada del famoso juego BlackJack o 21. Para esto, se tienen las siguientes reglas:


1. Solamente es un jugador y el Dealer, y en cada juagada se apuesta una cantidad fija de dinero (se requiere una política por jugada y los juegos son independientes)

2. Se asume un mazo infinito (todas las cartas tienen la misma probabilidad de salir)

3. El jugador juega primero, y solo tiene dos jugadas posibles: parar o pedir otra carta.

4. El jugador puede pedir cartas hasta que: llegue a 21 puntos (gana automáticamente), se pase de 21 puntos (pierde automáticamente), tenga 4 cartas o más (gana automáticamente) o decida parar.

5. El Dealer empieza con una sola carta y una vez que el jugador decide parar, y no haya perdido o ganado automáticamente.

6. El Dealer tiene que pedir_carta siempre que tenga menos de 17 puntos.

7. El Dealer tiene que parar si tiene 17 puntos o más.

8. Si el Dealer tiene 4 cartas, o un número mayor que el jugador (y menos que 21) gana. De otra forma, gana el jugador.


El modelo que se utilizará es el siguiente:

$ S = (d,sum,n,as)$ 

$ A = \{ fin, otra \} $

$ \rho = ...$ 

$ S_f =  \{ (0,0,0,1), (0,0,0,-1) \} $

$ r = $ 1 si gana, -1 si pierde y 0 en otro caso




d : es el valor que tiene la primer carta del dealer. La que le es mostrada al jugador

sum : es la suma acumulada que tiene el jugador hasta el momento. En caso de tener As se suman 1's.

n: es el numero de cartas que tiene hasta el momento.

as: indica con un 1 que tiene al menos un As, y 0 si no tiene ninguno.

Las acciones que se toman son, como se dijo en las reglas, finalizar su juego (fin) o pedir otra carta (otra).

Existen dos estados , los finales, de los cuales las componentes no tienen un significado en particular. Solo se utilizaron estos para representar si se gana (0,0,0,1) o si se pierde (0,0,0,-1)


## Definiciones principales del Modelo

In [1]:
abstract type MDP end
abstract type Estado end
abstract type Accion end

In [2]:
struct BJ_s <:Estado
    d #valor de primer carta del dealer
    sum #suma del jugador hasta el momento
    n #numero de cartas del jugador
    as #indica si se puede sumar 10 a sum o no (1,0), es decir, si tiene un As.

    function BJ_s(d::Int,sum::Int,n::Int,as::Int)
        if (d > 10 || sum>22 || n>4 || as>1 )
            error("estado invalido")
        else
            return new(d,sum,n,as)
        end
    end
end


In [3]:
BJ_s(5,0,0,-1)

BJ_s(5, 0, 0, -1)

In [4]:
struct BJ_a <:Accion # Se puede pedir otra carta o finalizar el juego.
    a
    function BJ_a(a)
        if (a != :fin && a!= :otra)
            error("accion invalida")
        else
            new(a)
        end
    end
end

In [5]:
BJ_a(:otra)

BJ_a(:otra)

## Funciones del Modelo

A continuación se presentan las funciones principales del modelo. 
1. La función que indica si un estado es terminal o no.
2. La función que devuelve todas las acciones legales en un estado.
3. La función de recompensa, la cual regresa un número positivo o negativo el cual califica cada transición que se hace, es decir, el estado en el que se está y al que se pasa despues de tomar una acción. 

In [37]:
#terminal
function terminal(s::BJ_s)
    (s.d==s.sum==s.n==0) && (s.as==1 || s.as==-1) ? true : false
end


#acciones legales
function aLegales(s::BJ_s)
    if terminal(s)
        return nothing
    end
    
    acciones = []
    
    if s.sum<=21 && s.n<4
        push!(acciones,BJ_a(:otra))
    end
    
    push!(acciones,BJ_a(:fin))
    
    acciones
end
                        
#recompensa
function r(s::BJ_s,a::BJ_a,s2::BJ_s)
    if terminal(s) return r(s) end
    0
end

function r(s::BJ_s)
    if s.d==s.sum==s.n==0 && s.as==1
        return 1
    elseif s.d==s.sum==s.n==0 && s.as==-1
        return -1
    end
    0
end


r (generic function with 2 methods)

In [38]:
# Funciones auxiliares

"""
Calcula el numero de opciones de una carta cuya valor esta entre dos valores. (izq_ , der_).
(Nota: El valor que se busca es estrictamente mayor que el izq_ y menor o igual que el der_)
"""

function comb1carta(izq_,der_)
    
    if izq_>= der_ return 0 end
    
    total=0;

    
    if izq_<9 total+=min(9,der_)-max(izq_,0) end
    if (der_>=10 && izq_<10) total+=4 end
    if (der_>=11 && izq_<11) total+=1 end
    

    #println("total de combinaciones ",total)
    total
end


"""
Calcula el numero de combinaciones de dos cartas cuya suma esta entre dos valores. (izq_ , der_)
(Nota: El valor suma que se busca es estrictamente mayor que el izq_ y menor o igual que el der_)
"""

function comb2cartas(izq_,der_,acum,cond)
        
    if izq_>= der_ return 0 end
            
    total=0; i=1 
    izq=izq_-i
    der=der_-i
    
    while (der>0 && i<=11 && acum+i<cond)
    
        y=comb1carta(izq,der)
        i==10 ? total+=4*min(y,13) : total+=min(y,13)
        i+=1
        izq-=1
        der-=1
    end
    #println("total de combinaciones ",total)
    total
end

"""
Calcula el numero de combinaciones de tres cartas cuya suma esta entre dos valores. (izq_ , der_)
(Nota: El valor suma que se busca es estrictamente mayor que el izq_ y menor o igual que el der_)
"""

function comb3cartas(izq_,der_, acum, cond)
        
    if izq_>= der_ return 0 end
    total=0
    for i in 1:11
        
        if acum + i >=17 break end
            
        total+=comb2cartas(izq_ - i, der_ -i, acum+i, cond)
    end
    total
end



La función de probabilidad fue la función más complicada de desarrollar en este problema. Intenté abarcar todos los casos sin embargo no consideré cuando el dealer tiene un As como primera carta. Aún así en los resultados finales no parece afectar mucho la cantidad que tiene el dealer si no la cantidad acumulada del jugador que era lo que se esperaba. Al final se tiene una estrategia parecida a la del dealer.

In [39]:
#probabilidad de Transición
"""
Calcula la probabilidad de partir de un estado s a un estado s2 tomando la accion a.
"""
function ρ(s::BJ_s,a::BJ_a,s2::BJ_s)
    
    if  terminal(s) 
        return 0
    end
    
    if a ∉ aLegales(s)
        return 0
    end
    
    if a.a == :fin
        
        if s2.d==s2.sum==s2.n==0 && s2.as==1 #si gana
            s.as==1 && s.sum+10< 22 ? sum=s.sum+10 : sum=s.sum #de una vez sumo lo del as si conviene.
            if sum==21 return 1 end
            
            
            z=0 #todos los posibles juegos del dealer
            
            z+=comb1carta(16-s.d,44-s.d) #Jugadas con 2 cartas
            z+=comb2cartas(16-s.d,44-s.d,s.d,17)#Jugadas con 3 cartas
            z+=comb3cartas(16-s.d,44-s.d,s.d,17)#Jugadas con 4 cartas 
            z+=comb3cartas(0,44-s.d,s.d,17) #Jugadas con 4 cartas
    
            x=0 #guarda todos los casos que se necesiten contar. En que pierda el dealer o gane, segun se explica abajo.
            
            if sum >= s.d # si el jugador tiene mas que la carta del dealer
                dif = sum-s.d
                
                #Formas en las que puede perder el dealer (en general): 
                #con 2 cartas 
                """if dif<10 
                    x+=dif-1
                    if s.d+11 >21 x+=1 end
                elseif dif==10 x+=12 
                else x+=13
                end"""
                
                #con 3 cartas (en general)
                """
                x+=169 - comb2cartas(dif,21-s.d)
                """
                #con 4 cartas (en general)
                """
                x+=2197 - comb3cartas(dif,21-s.d)
                
                return x/2379"""
                
                #Formas en las que puede ganar el dealer (jugando con las reglas de dealer).
                #con 2 cartas : max(16,sum)<s.d + 1 carta<=21.
                #con 3 cartas: max(16,sum)<s.d + suma de 2 cartas<=21.
                #con 4 cartas: max(16,sum)<s.d + suma de 3 cartas<=21 o tambien que la suma sea menor que 21.
                
                if s.d!=1
                    x+=comb1carta(max(16,sum)-s.d,21-s.d) #Casos en los que gana con 2 cartas 
                    x+=comb2cartas(max(16,sum)-s.d,21-s.d,s.d,17)#Casos en los que gana con 3 cartas 
                    x+=comb3cartas(max(16,sum)-s.d,21-s.d,s.d,17)#Casos en los que gana con 4 cartas 
                    x+=comb3cartas(0,21-s.d,s.d,17) #Casos en los que gana con 4 cartas 
                else
                    x+=comb1carta(max(16,sum)-s.d,21-s.d) #Casos en los que gana con 2 cartas 
                    x+=comb2cartas(max(16,sum)-s.d,21-s.d,s.d,17)#Casos en los que gana con 3 cartas 
                    x+=comb3cartas(max(16,sum)-s.d,21-s.d,s.d,17)#Casos en los que gana con 4 cartas 
                    x+=comb3cartas(0,21-s.d,s.d,17) #Casos en los que gana con 4 cartas 
                end

  
                return 1 - x/z
                        
            else # si el dealer inicialmente ya tiene mas que el jugador.
                        
                # La probabilidad de que el dealer se pase (en general)
                """
                x+=169 - comb2cartas(s.d,21)
                x+=2197 - comb3cartas(s.d,21)
                
                return x/2380
                """
                # La probabilidad de que el dealer se pase (reglas del dealer)
                if s.d!=1
                    x+=comb2cartas(21-s.d,44-s.d,s.d,17)#Casos en los que se pasa con 3 cartas
                    x+=comb3cartas(21-s.d,44-s.d,s.d,17)#Casos en los que se pasa con 4 cartas
                else
                    x+=comb2cartas(21-s.d,44-s.d,s.d,17)#Casos en los que se pasa con 3 cartas 
                    x+=comb3cartas(21-s.d,44-s.d,s.d,17)#Casos en los que se pasa con 4 cartas
                end
                
                return x/z
            
            end
            
            
            
        elseif s2.d==s2.sum==s2.n==0 && s2.as==-1 #si pierde
            return 1 - ρ(s,a,BJ_s(0,0,0,1))
        
        #else: pasar a otro estado que no sea ganar ni perder = 0
        end
        
    elseif a.a== :otra
        
        if s2.d==s2.sum==s2.n==0 && s2.as==1 #si gana automaticamente. (llegar a las 4 cuartas y sum<21)
            if s.n==3
                dif=21-s.sum
                if dif<10 return (dif/13) #no me dejo dif<10 ? return dif/13:1
                else return 1
                end
            #else: cundo no tendra 4 cartas e inclusive puede tener 21. Solo que necesita una jugada 
            #extra donde decida finalizar
            end
        elseif s2.d==s2.sum==s2.n==0 && s2.as==-1 #si pierde automaticamente.(cuando se pase de 21)
            dif=22-s.sum
            if dif<10  return (10-dif+4)/13
            elseif dif==10 return 4/13
            #else: cuando la diferencia sea mas de lo que puede sumar una carta
            end
        elseif s2.d ==s.d && s2.n==s.n+1 #si aumenta en 1 el num de cartas y el dealer es el mismo
                if s2.sum-s.sum<10 && s2.sum-s.sum>1 #si anda entre 2 y 9 lo que aumenta la suma
                    return 0.0769 #1/13
                elseif s2.sum-s.sum==1 # si aumento en 1 la suma, solo puede ser un As
                    if s2.as==1 return 0.0769 end                    
                elseif s2.sum-s.sum==10 #si aumenta en 10 hay 4 posibilidades
                    return 0.3077
                end 
        end
    
    end
    
    0 #todos los else
        
end

ρ (generic function with 1 method)

In [41]:
s1=BJ_s(8,15,2,0)
s2=BJ_s(4, 11, 3, 1)
s3=BJ_s(0,0,0,1)

println("Es terminal el estado $s1 ? : ",terminal(s1))
println("Acciones legales de $s1 : ", aLegales(s1))
println("Acciones legales de $s2 : ", aLegales(s2))
println("Acciones legales de $s3 : ", repr(aLegales(s3)))

a=BJ_a(:otra)

println("Recompenza de $s1 : ", r(s1,a,s2))

println("Probabilidad de pasar de $s2 a $s3 tomando la accion $a: ",ρ(s2,a,s3))



Es terminal el estado BJ_s(8, 15, 2, 0) ? : false
Acciones legales de BJ_s(8, 15, 2, 0) : Any[BJ_a(:otra), BJ_a(:fin)]
Acciones legales de BJ_s(4, 11, 3, 1) : Any[BJ_a(:otra), BJ_a(:fin)]
Acciones legales de BJ_s(0, 0, 0, 1) : nothing
Recompenza de BJ_s(8, 15, 2, 0) : 0
Probabilidad de pasar de BJ_s(4, 11, 3, 1) a BJ_s(0, 0, 0, 1) tomando la accion BJ_a(:otra): 1


La probabilidad de ganar estando en el estado s2 tomando otra carta es 1 debido a que no importa qué carta saque, tendrá menos o exactamente 21, lo que lo hará ganar.

## Generacion de estados

Enseguida se generan todos los estados, tratando de evadir los que no tienen sentido.

In [ ]:
est=[BJ_s(1,1,1,0)]
pop!(est)

for k in 2:4 #num
    if k==1
        x=10 
    elseif k==2 
        x=20
    else
        x=21
    end
    for i in 1:10 #dealer
        for j in 1:x #sum
            for l in 0:1 #as
                if j>k
                    if l==1 && j<=(k-1)*10
                        push!(est,BJ_s(i,j,k,l))
                    elseif l==0
                        push!(est,BJ_s(i,j,k,l))
                    end
                end
            end
        end
    end
end

push!(est,BJ_s(0,0,0,1))
push!(est,BJ_s(0,0,0,-1))

est

## Generación de acciones

In [43]:
ac = [BJ_a(:fin), BJ_a(:otra)]
ac

2-element Array{BJ_a,1}:
 BJ_a(:fin) 
 BJ_a(:otra)

## Modelo MDP

In [44]:
struct BJ <: MDP
    estados
    acciones
    function BJ(estados::Array{BJ_s,1}, acciones::Array{BJ_a,1})
        new(estados,acciones)
    end
end

In [45]:
BlackJ = BJ(est,ac)


BJ(BJ_s[BJ_s(1, 3, 2, 0), BJ_s(1, 3, 2, 1), BJ_s(1, 4, 2, 0), BJ_s(1, 4, 2, 1), BJ_s(1, 5, 2, 0), BJ_s(1, 5, 2, 1), BJ_s(1, 6, 2, 0), BJ_s(1, 6, 2, 1), BJ_s(1, 7, 2, 0), BJ_s(1, 7, 2, 1)  …  BJ_s(10, 18, 4, 0), BJ_s(10, 18, 4, 1), BJ_s(10, 19, 4, 0), BJ_s(10, 19, 4, 1), BJ_s(10, 20, 4, 0), BJ_s(10, 20, 4, 1), BJ_s(10, 21, 4, 0), BJ_s(10, 21, 4, 1), BJ_s(0, 0, 0, 1), BJ_s(0, 0, 0, -1)], BJ_a[BJ_a(:fin), BJ_a(:otra)])

## Algoritmo de Programación Dinámica

In [50]:
function policy_value(mdp::MDP,π_::Dict, γ::Real,ϵ::Float64)
    
    V = Dict(s => 0.0 for s in mdp.estados)
    
    flag=true
    
    Δ=ϵ+1
    while Δ > ϵ
        Δ=0
        for s ∈ keys(V)
            
            v=V[s]
            
            if terminal(s) 
                V[s]=r(s)
            else
                V[s]=sum([ρ(s,π_[s],sp)*(r(s,π_[s],sp)+γ*V[sp]) for sp ∈ keys(V)] )
            end
            Δ=maximum([Δ,abs(v-V[s])])
            
        end
    end
    V
end

policy_value (generic function with 1 method)

In [51]:
function policy_value_(mdp::MDP,π_::Dict, γ::Real)
    
    v = Dict(si => 0.0 for si in mdp.estados)
    
    flag=true
    
    while flag
        
        flag=false
        
        for s ∈ keys(π_)
            if terminal(s) v[s]=r(s) 
            else
                temp=sum([ρ(s,π_[s],sp)*(r(s,π_[s],sp)+γ*v[sp]) for sp in keys(v)])

                if temp!=v[s]
                    flag=true
                    v[s]=temp
                end
                
            end
        end
    end
    
    v
end

policy_value_ (generic function with 1 method)

In [52]:
function policy_iteration(mdp::MDP,γ::Real, ϵ::Float64)
    
    
    π_ = Dict(s => rand(aLegales(s)) for s in mdp.estados if !terminal(s))
    
    optima=false
   
    while !optima
        v = policy_value(mdp,π_,γ,ϵ)
                    
        optima=true

        for s ∈ keys(π_)
            for a ∈ aLegales(s)
                temp=sum( [ρ(s,a,s2)*(r(s,a,s2)+γ*v[s2]) for s2 in keys(v)])
                if temp > v[s] 
                    optima=false      
                    π_[s]=a
                end
            end
        end
    end
    
    π_
end

policy_iteration (generic function with 1 method)

In [53]:
π_opt= policy_iteration(BlackJ,1,0.0)


Dict{BJ_s,BJ_a} with 950 entries:
  BJ_s(8, 16, 3, 1)  => BJ_a(:otra)
  BJ_s(3, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 8, 4, 0)   => BJ_a(:fin)
  BJ_s(9, 10, 4, 0)  => BJ_a(:fin)
  BJ_s(7, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 15, 4, 0)  => BJ_a(:fin)
  BJ_s(3, 15, 3, 1)  => BJ_a(:otra)
  BJ_s(5, 18, 3, 0)  => BJ_a(:fin)
  BJ_s(7, 9, 4, 1)   => BJ_a(:fin)
  BJ_s(6, 10, 2, 0)  => BJ_a(:otra)
  BJ_s(6, 16, 3, 0)  => BJ_a(:otra)
  BJ_s(10, 8, 4, 0)  => BJ_a(:fin)
  BJ_s(2, 13, 2, 0)  => BJ_a(:fin)
  BJ_s(10, 13, 3, 1) => BJ_a(:otra)
  BJ_s(8, 6, 2, 0)   => BJ_a(:otra)
  BJ_s(10, 4, 3, 1)  => BJ_a(:otra)
  BJ_s(2, 14, 4, 1)  => BJ_a(:fin)
  BJ_s(4, 12, 3, 0)  => BJ_a(:otra)
  BJ_s(2, 4, 2, 0)   => BJ_a(:otra)
  BJ_s(9, 21, 3, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 4, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 3, 1)  => BJ_a(:otra)
  BJ_s(3, 12, 2, 0)  => BJ_a(:otra)
  BJ_s(9, 8, 4, 1)   => BJ_a(:fin)
  BJ_s(8, 9, 4, 1)   => BJ_a(:fin)
  ⋮                  => ⋮

In [54]:
muestra=rand(π_opt,20)

for i ∈ 1:20
    println(muestra[i])
end

BJ_s(5, 7, 3, 0) => BJ_a(:otra)
BJ_s(9, 16, 3, 1) => BJ_a(:otra)
BJ_s(4, 17, 2, 0) => BJ_a(:fin)
BJ_s(9, 17, 3, 0) => BJ_a(:fin)
BJ_s(3, 9, 2, 1) => BJ_a(:otra)
BJ_s(2, 16, 4, 1) => BJ_a(:fin)
BJ_s(2, 5, 4, 0) => BJ_a(:fin)
BJ_s(4, 8, 4, 1) => BJ_a(:fin)
BJ_s(10, 13, 2, 0) => BJ_a(:fin)
BJ_s(6, 6, 4, 0) => BJ_a(:fin)
BJ_s(5, 8, 3, 1) => BJ_a(:otra)
BJ_s(7, 19, 2, 0) => BJ_a(:fin)
BJ_s(1, 3, 2, 1) => BJ_a(:otra)
BJ_s(6, 12, 4, 1) => BJ_a(:fin)
BJ_s(6, 13, 4, 1) => BJ_a(:fin)
BJ_s(1, 11, 4, 1) => BJ_a(:fin)
BJ_s(2, 10, 4, 1) => BJ_a(:fin)
BJ_s(6, 10, 3, 0) => BJ_a(:otra)
BJ_s(2, 6, 2, 1) => BJ_a(:otra)
BJ_s(10, 11, 3, 0) => BJ_a(:otra)


In [55]:
function iter_value(mdp::MDP,γ::Real)
    
    V = Dict(s => 0.0 for s in mdp.estados)
    Vp = Dict(s => 0.0 for s in mdp.estados)
    
    optima=false
    
    while !optima
        
        optima = true
        i=0
        for s in keys(V)
            
            if terminal(s)
                V[s]=r(s)
            else
                Vp[s]=maximum([sum([ ρ(s,a,s2)==0 ? r(s) : ρ(s,a,s2)*(r(s,a,s2)+γ*V[s2]) for s2 in mdp.estados ]) 
                        for a in aLegales(s)])
                
                if Vp[s]>V[s] 
                    optima=false
                    V[s]=Vp[s]
                end
                
            end
            
        end
        
        
    end
    
    s_legales=[s for s in mdp.estados if !terminal(s)]
    sx=rand(s_legales)

    π_ = Dict(sx => rand(aLegales(sx)))
    
    for s in s_legales 
        dict = Dict(a => sum([ρ(s,a,s2)*(r(s,a,s2)+γ*V[s2]) for s2 in mdp.estados]) for a in aLegales(s))
        π_[s]=findmax(dict)[2]
    end
    
    π_ 
    
end

iter_value (generic function with 1 method)

In [56]:
π_opt= iter_value(BlackJ,1)


Dict{BJ_s,BJ_a} with 950 entries:
  BJ_s(8, 16, 3, 1)  => BJ_a(:otra)
  BJ_s(3, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 8, 4, 0)   => BJ_a(:fin)
  BJ_s(9, 10, 4, 0)  => BJ_a(:fin)
  BJ_s(7, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 15, 4, 0)  => BJ_a(:fin)
  BJ_s(3, 15, 3, 1)  => BJ_a(:otra)
  BJ_s(5, 18, 3, 0)  => BJ_a(:fin)
  BJ_s(7, 9, 4, 1)   => BJ_a(:fin)
  BJ_s(6, 10, 2, 0)  => BJ_a(:otra)
  BJ_s(6, 16, 3, 0)  => BJ_a(:otra)
  BJ_s(10, 8, 4, 0)  => BJ_a(:fin)
  BJ_s(2, 13, 2, 0)  => BJ_a(:otra)
  BJ_s(10, 13, 3, 1) => BJ_a(:otra)
  BJ_s(8, 6, 2, 0)   => BJ_a(:otra)
  BJ_s(10, 4, 3, 1)  => BJ_a(:otra)
  BJ_s(2, 14, 4, 1)  => BJ_a(:fin)
  BJ_s(4, 12, 3, 0)  => BJ_a(:otra)
  BJ_s(2, 4, 2, 0)   => BJ_a(:otra)
  BJ_s(9, 21, 3, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 4, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 3, 1)  => BJ_a(:otra)
  BJ_s(3, 12, 2, 0)  => BJ_a(:otra)
  BJ_s(9, 8, 4, 1)   => BJ_a(:fin)
  BJ_s(8, 9, 4, 1)   => BJ_a(:fin)
  ⋮                  => ⋮

In [57]:
muestra=rand(π_opt,20)

for i ∈ 1:20
    println(muestra[i])
end

BJ_s(7, 18, 2, 0) => BJ_a(:fin)
BJ_s(7, 10, 2, 0) => BJ_a(:otra)
BJ_s(5, 21, 4, 1) => BJ_a(:fin)
BJ_s(8, 10, 4, 1) => BJ_a(:fin)
BJ_s(6, 20, 4, 1) => BJ_a(:fin)
BJ_s(4, 17, 4, 0) => BJ_a(:fin)
BJ_s(2, 16, 4, 1) => BJ_a(:fin)
BJ_s(4, 20, 3, 1) => BJ_a(:fin)
BJ_s(2, 10, 2, 0) => BJ_a(:otra)
BJ_s(2, 19, 4, 1) => BJ_a(:fin)
BJ_s(1, 7, 2, 1) => BJ_a(:otra)
BJ_s(2, 15, 4, 1) => BJ_a(:fin)
BJ_s(9, 17, 3, 1) => BJ_a(:fin)
BJ_s(10, 6, 3, 1) => BJ_a(:otra)
BJ_s(6, 21, 3, 0) => BJ_a(:fin)
BJ_s(4, 10, 4, 0) => BJ_a(:fin)
BJ_s(10, 18, 3, 1) => BJ_a(:fin)
BJ_s(5, 15, 3, 1) => BJ_a(:otra)
BJ_s(7, 19, 4, 1) => BJ_a(:fin)
BJ_s(4, 12, 3, 1) => BJ_a(:otra)


Algunos resultados de nuestra política óptima son:

Si tiene 4 cartas elige 'fin' porque es la unica accion legal.

Si tiene como suma 17,18 o 20 y puede pedir otra elige 'fin'.

Cuando tiene menos que 17 y puede pedir otra carta al parecer la pide. Sin embargo en algunas ocasiones me encontré con situaciones donde decidía finalizar lo que me hace reconsiderar que la función de probabilidad aún falta ajustarse.